In [76]:
import requests
import os
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td
from dateutil.relativedelta import relativedelta
import json
import time

In [78]:
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [60]:
UUIDS = {
    'bhadra':'767d51c7-7f38-440b-8857-397ec7633c0a',
    'harangi':'6c4ae6c1-8ed0-4f27-a70a-a69e0a3ca717',
    'hemavathy':'e8760779-2e62-4779-8a7c-c6fd98de644e',
    'kabini':'537d7bac-1cb6-4cd8-a86a-80f9ab484fe0',
    'krishnaraja_sagar':'801a7d3c-492e-4cbf-a2c9-cbf19d8781b0',
    'lower_bhawani':'fdaac965-b143-4714-b74e-d7d75f6b3572',
}

In [61]:
sd = dt(2021,5,1)
ed = dt(2021,9,13)

In [71]:
sd + relativedelta(month=3)

datetime.datetime(2021, 3, 1, 0, 0)

In [81]:
def get_reservoir_data(sdate, edate, reservoir_code):

    headers = {
        'Content-Type': 'application/json',
    }
    url = 'http://wdo.indiawris.gov.in/api/reservoir/chart'
    payload = {
        "eDate": edate.strftime('%Y%m%d'), 
        "format": "yyyyMMdd", 
        "lType": "STATION", 
        "pUUID": reservoir_code, #00cb31c7-2928-4e6d-adcc-443d478c90d6
        "sDate": sdate.strftime('%Y%m%d'), 
        "src": "CWC"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    data = json.loads(response.text)
    
    df = pd.DataFrame(index=data['labels'], data={vv['label']:vv['data'] for vv in data['data']})
    
    print (sdate.isoformat(),edate.isoformat(),reservoir,'STATUS:',response.status_code, 'DF:',len(df))
    
    return df

In [91]:
sd = dt(2010,1,1)
while sd<dt(2012,1,1):
    
    ed = sd+relativedelta(months=3)
    
    print ('DOING', sd.isoformat(), ed.isoformat())
    
    for reservoir,code in UUIDS.items():
        
        df = get_reservoir_data(sd,ed,code)
        
        df.to_csv(os.path.join(root,'data',f'{reservoir}_{sd.isoformat()}_{ed.isoformat()}.csv'))
        
        time.sleep(3)
        
    sd = sd + relativedelta(months=3)

DOING 2010-01-01T00:00:00 2010-04-01T00:00:00
2010-01-01T00:00:00 2010-04-01T00:00:00 bhadra STATUS: 200 DF: 91
2010-01-01T00:00:00 2010-04-01T00:00:00 harangi STATUS: 200 DF: 91
2010-01-01T00:00:00 2010-04-01T00:00:00 hemavathy STATUS: 200 DF: 91
2010-01-01T00:00:00 2010-04-01T00:00:00 kabini STATUS: 200 DF: 91
2010-01-01T00:00:00 2010-04-01T00:00:00 krishnaraja_sagar STATUS: 200 DF: 91
2010-01-01T00:00:00 2010-04-01T00:00:00 lower_bhawani STATUS: 200 DF: 91
DOING 2010-04-01T00:00:00 2010-07-01T00:00:00
2010-04-01T00:00:00 2010-07-01T00:00:00 bhadra STATUS: 200 DF: 92
2010-04-01T00:00:00 2010-07-01T00:00:00 harangi STATUS: 200 DF: 92
2010-04-01T00:00:00 2010-07-01T00:00:00 hemavathy STATUS: 200 DF: 92
2010-04-01T00:00:00 2010-07-01T00:00:00 kabini STATUS: 200 DF: 92
2010-04-01T00:00:00 2010-07-01T00:00:00 krishnaraja_sagar STATUS: 200 DF: 92
2010-04-01T00:00:00 2010-07-01T00:00:00 lower_bhawani STATUS: 200 DF: 92
DOING 2010-07-01T00:00:00 2010-10-01T00:00:00
2010-07-01T00:00:00 2010-1

In [92]:
from google.cloud import bigquery

In [93]:
import glob

In [96]:
fs = sorted(glob.glob(os.path.join(root,'data','*.csv')))

In [98]:
pd.read_csv(fs[0])

,Unnamed: 0,Current Year Level,Current Year Storage,Last Year Storage,Last 10 Year Average Storage,Last Year Level,Last 10 Year Average Level,Full Reservoir Level
0,20100101,657.75,1.785,1.580,1.3221,655.95,653.028,657.76
1,20100102,657.75,1.785,1.582,1.3190,655.95,652.991,657.76
2,20100103,657.75,1.785,1.582,1.3160,655.98,652.962,657.76
3,20100104,657.75,1.785,1.582,1.3126,655.98,652.918,657.76
4,20100105,657.75,1.785,1.582,1.3089,655.98,652.881,657.76
...,...,...,...,...,...,...,...,...
86,20100328,652.48,1.218,0.833,0.7233,648.12,646.146,657.76
87,20100329,652.39,1.210,0.825,0.7175,648.02,646.068,657.76
88,20100330,652.30,1.201,0.817,0.7097,647.97,645.973,657.76
89,20100331,652.22,1.192,0.809,0.7027,647.82,645.868,657.76


In [102]:
bqclient.insert_rows_json(table_name, [vals])

'2020-05-06'

In [103]:
client=bigquery.Client()

In [111]:
table_name = 'oxeo-main.wave2web.historic'

In [114]:
a = 'aaa_bb_cc_dd'

In [117]:
'_'.join(a.split('_')[:-2])

'aaa_bb'

In [119]:
for f in fs:
    print (f)
    reservoir = '_'.join(os.path.split(f)[1].split('_')[:-2])
    df = pd.read_csv(f).set_index('Unnamed: 0')
    
    entries = []
    for idx, row in df.iterrows():
        entries.append({
            'reservoir':reservoir,
            'date':dt.strptime(str(idx),'%Y%m%d').isoformat()[0:10],
            'volume_bcm':float(row['Current Year Storage']),
        })       
    
    errors = client.insert_rows_json(table_name,entries)

/home/jupyter/era5-gcp/data/bhadra_2010-01-01T00:00:00_2010-04-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2010-04-01T00:00:00_2010-07-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2010-07-01T00:00:00_2010-10-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2010-10-01T00:00:00_2011-01-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2011-01-01T00:00:00_2011-04-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2011-04-01T00:00:00_2011-07-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2011-07-01T00:00:00_2011-10-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2011-10-01T00:00:00_2012-01-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2012-01-01T00:00:00_2012-04-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2012-04-01T00:00:00_2012-07-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2012-07-01T00:00:00_2012-10-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2012-10-01T00:00:00_2013-01-01T00:00:00.csv
/home/jupyter/era5-gcp/data/bhadra_2013-01-01T00:00:

In [120]:
forecast_data

NameError: name 'forecast_data' is not defined

In [ ]:
        vals = {
            "job_id": self["job_id"],
            "task_id": self.task_id,
            "storage_gs_path": self["storage_gs_path"],
            "msg_type": msg_type,
            "msg_payload": msg_payload,
            "timestamp": datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f"),
        }

        errors = bqclient.insert_rows_json(table_name, [vals])

In [63]:
for kk in UUIDS:
    df = get_reservoir_data(sd,ed,kk)

200 59
200 49
200 49
200 49
200 48
200 55


In [57]:
data['data'][0]['label']

'Current Year Level'

,Current Year Level,Current Year Storage,Last Year Storage,Last 10 Year Average Storage,Last Year Level,Last 10 Year Average Level,Full Reservoir Level
20210503,688.705,0.094,0.187,0.081400,691.104,688.383600,696.16
20210504,688.650,0.092,0.185,0.080100,691.067,688.344500,696.16
20210505,688.598,0.090,0.184,0.079000,691.033,688.305300,696.16
20210506,688.580,0.089,0.182,0.078300,690.997,688.267700,696.16
20210511,688.400,0.083,0.168,0.075800,690.646,688.138700,696.16
20210512,688.382,0.083,0.165,0.075100,690.579,688.108900,696.16
20210515,688.513,0.087,0.154,0.074200,690.308,688.038100,696.16
20210516,689.360,0.117,0.149,0.076300,690.198,688.086000,696.16
20210517,689.820,0.134,0.145,0.077400,690.082,688.096400,696.16
20210518,689.921,0.138,0.140,0.077500,689.976,688.079800,696.16


In [40]:
json.loads(r.text)

JSONDecodeError: Expecting value: line 3 column 1 (char 2)

In [ ]:
response = requests.post('http://wdo.indiawris.gov.in/api/reservoir/chart', headers=headers, data=data)